In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
import urllib
import requests
import re

# Ongoing problems:
I'm not exactly sure what was causing some of my previous sparql queries to fail, but I think I have pulled down basic information for all the papers in the ma graph database. I'm not sure how useful this is going to be. I'm now getting some details from around 3M pubmed papers that are hosted by a research group at [NCBI](https://www.ncbi.nlm.nih.gov/research/bionlp/APIs/BioC-PMC/). I would ideally like to filter to just papers after 2008 or so (when github was founded), but I'm not sure that is possible. So instead, I'm doing one pass through all the listed files collecting minimal information. Then, any further data needed can be queried on the subset that I'm interested in.

## Data description
Here, I'm pulling data out of a sparql database that includes information from microsoft academic knowledge graph. The website for this data is here: http://ma-graph.org/


In [3]:
sparql = SPARQLWrapper('http://ma-graph.org/sparql')


In [97]:
sparql.setQuery("""
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX magp: <http://ma-graph.org/property/>
PREFIX mag: <http://ma-graph.org/>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX fabio: <http://purl.org/spar/fabio/>
PREFIX org: <http://www.w3.org/ns/org#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX cito: <http://purl.org/spar/cito/>
PREFIX c4o: <http://purl.org/spar/c4o/>
PREFIX prism: <http://prismstandard.org/namespaces/1.2/basic/>
PREFIX datacite: <http://purl.org/spar/datacite/>

SELECT ?nihauthname ?otherauths ?papertitle ?discname ?dt ?doi ?abst
WHERE {
?inst rdf:type <http://ma-graph.org/class/Affiliation> .
?inst foaf:name "National Institutes of Health"^^xsd:string .
?auth org:memberOf ?inst .
?paper dcterms:creator ?auth .
?auth foaf:name ?nihauthname .
?paper dcterms:creator ?allauths .
?allauths foaf:name ?otherauths .
?paper dcterms:title ?papertitle .
?paper prism:publicationDate ?dt .
?paper fabio:hasDiscipline ?disc .
?paper datacite:doi ?doi .
?paper dcterms:abstract ?abst .
?disc foaf:name ?discname .
}
""")

In [98]:
#?paper cito:hasCitedEntity ?cit .
#?cit c4o:hasContext ?citcont .

In [99]:
sparql.setReturnFormat(JSON)
results = sparql.query().convert()


In [100]:
len(results['results']['bindings'])

1048576

In [106]:
results['results']['bindings'][10000]

{'nihauthname': {'type': 'typed-literal',
  'datatype': 'http://www.w3.org/2001/XMLSchema#string',
  'value': 'Scott R. Budihas'},
 'otherauths': {'type': 'typed-literal',
  'datatype': 'http://www.w3.org/2001/XMLSchema#string',
  'value': 'Michael A. Parniak'},
 'papertitle': {'type': 'typed-literal',
  'datatype': 'http://www.w3.org/2001/XMLSchema#string',
  'value': 'A capillary electrophoretic assay for ribonuclease H activity.'},
 'discname': {'type': 'typed-literal',
  'datatype': 'http://www.w3.org/2001/XMLSchema#string',
  'value': 'Biochemistry'},
 'dt': {'type': 'typed-literal',
  'datatype': 'http://www.w3.org/2001/XMLSchema#date',
  'value': '2004-08-01'},
 'doi': {'type': 'typed-literal',
  'datatype': 'http://www.w3.org/2001/XMLSchema#string',
  'value': '10.1016/j.ab.2004.05.017'},
 'abst': {'type': 'typed-literal',
  'datatype': 'http://www.w3.org/2001/XMLSchema#string',
  'value': 'Abstract A capillary electrophoretic assay was developed to measure the ribonuclease (RN

In [82]:
outdat = []
for i in results['results']['bindings']:
    outdat.append([i['papertitle']['value'], i['authname']['value'], 
                   i['discname']['value'], i['dt']['value'], i['doi']['value']])

In [83]:
dat = pd.DataFrame(outdat, columns = ['title', 'author', 'field', 'date', 'doi'])

In [92]:
dat[dat.doi=='10.1128/genomeA.01093-15']

,title,author,field,date,doi
0,Molecular Signature of the Ebola Virus Associa...,Concetta Castilletti,Ebola virus,2015-10-29,10.1128/genomeA.01093-15
1,Molecular Signature of the Ebola Virus Associa...,Concetta Castilletti,Virology,2015-10-29,10.1128/genomeA.01093-15
2,Molecular Signature of the Ebola Virus Associa...,Concetta Castilletti,Subclade,2015-10-29,10.1128/genomeA.01093-15
3,Molecular Signature of the Ebola Virus Associa...,Concetta Castilletti,Outbreak,2015-10-29,10.1128/genomeA.01093-15
4,Molecular Signature of the Ebola Virus Associa...,Concetta Castilletti,Bioinformatics,2015-10-29,10.1128/genomeA.01093-15
5,Molecular Signature of the Ebola Virus Associa...,Concetta Castilletti,Biology,2015-10-29,10.1128/genomeA.01093-15
160224,Molecular Signature of the Ebola Virus Associa...,Silvia Meschi,Ebola virus,2015-10-29,10.1128/genomeA.01093-15
160225,Molecular Signature of the Ebola Virus Associa...,Silvia Meschi,Virology,2015-10-29,10.1128/genomeA.01093-15
160226,Molecular Signature of the Ebola Virus Associa...,Silvia Meschi,Subclade,2015-10-29,10.1128/genomeA.01093-15
160227,Molecular Signature of the Ebola Virus Associa...,Silvia Meschi,Outbreak,2015-10-29,10.1128/genomeA.01093-15


In [84]:
dat.groupby('field').count()[['title']].sort_values('title', ascending=False)

,title
field,
Biology,56725
Medicine,41832
Molecular biology,23265
Immunology,22135
Internal medicine,19022
Virology,15039
Biochemistry,14901
Endocrinology,13882
Genetics,13696


## URL presence

To what extent have open-data repositories been adopted? Have some repositories been adopted more quickly than others? Does this vary by field?

These questions have bearing on our understanding of the effects of open-data, and open science more generally. In principle, open science is designed to facilitate the participation of a wide range of people in the scientific community. If groups that have a disproportionate share of resources consistently publish their likely resource-intensive datasets, then these valuable datasets can be mined by a wider range of individuals for scientific insights. 

To this end, I will examine how frequently URL's to some prominent open science repositories are present in papers as urls

In [19]:
sparql.setQuery("""
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX magp: <http://ma-graph.org/property/>
PREFIX mag: <http://ma-graph.org/>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX fabio: <http://purl.org/spar/fabio/>
PREFIX org: <http://www.w3.org/ns/org#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX cito: <http://purl.org/spar/cito/>
PREFIX datacite: <http://purl.org/spar/datacite/>
PREFIX prism: <http://prismstandard.org/namespaces/1.2/basic/>
PREFIX c4o: <http://purl.org/spar/c4o/>

SELECT ?paper ?paptitle ?abst ?dt ?ln ?auth ?doi ?estCites
WHERE {
?paper rdf:type <http://ma-graph.org/class/Paper> .
?paper dcterms:title ?paptitle .
?paper dcterms:abstract ?abst .
?paper prism:publicationDate ?dt .
?paper dcterms:creator ?auth .
?paper datacite:doi ?doi .
?paper dcterms:language ?ln .
?paper dcterms:creator ?auth .
}
""")

In [20]:
sparql.setReturnFormat(JSON)
results = sparql.query().convert() 

In [21]:
len(results['results']['bindings'])

1048576

In [7]:
results['results']['bindings'][2]

{'paper': {'type': 'uri', 'value': 'http://ma-graph.org/entity/4524729'},
 'paptitle': {'type': 'typed-literal',
  'datatype': 'http://www.w3.org/2001/XMLSchema#string',
  'value': 'Fiber Optics Sensors'},
 'abst': {'type': 'typed-literal',
  'datatype': 'http://www.w3.org/2001/XMLSchema#string',
  'value': 'Fiber optic technology started in 1970\x19s, for long distance telecommunications, and it has experienced an exponential growth during the last four decades. Sensing applications are a small spin-off from this technology, taking profit of developments done for optoelectronic components and concepts. Optical fibers are cylindrical dielectric waveguides for the propagation of the light, made out from high purity, low loss optical materials, usually silica (Optical fibers made from polymers and other transparent materials are also commercially available). The refractive index of the core (about 1.46 for silica) is slightly higher than surrounding material or cladding, due to the prese

In [21]:
outdat = []
for i in results['results']['bindings']:
    outdat.append([i['paper']['value'], i['paptitle']['value'], 
                   i['abst']['value'], i['dt']['value'], i['ln']['value'],
                   i['doi']['value']])

In [22]:
dat = pd.DataFrame(outdat, columns = ['paper', 'title', 'abstract', 'date',
                                      'language', 'doi'])
dat = dat[dat.lang=='en']
dat['open_science'] = dat.abstract.str.contains('open science', case=False)

In [25]:
dat.to_pickle('/home/riddleta/ac_knowl/output/testdat.pkl')

In [28]:
sparql.setQuery("""
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX magp: <http://ma-graph.org/property/>
PREFIX mag: <http://ma-graph.org/>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX fabio: <http://purl.org/spar/fabio/>
PREFIX org: <http://www.w3.org/ns/org#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX cito: <http://purl.org/spar/cito/>
PREFIX datacite: <http://purl.org/spar/datacite/>
PREFIX prism: <http://prismstandard.org/namespaces/1.2/basic/>
PREFIX c4o: <http://purl.org/spar/c4o/>

SELECT ?author ?dt ?cites ?papes ?authname ?rnk ?inst
WHERE {
?author rdf:type <http://ma-graph.org/class/Author> .
}
""")

In [29]:
sparql.setReturnFormat(JSON)
results = sparql.query().convert() 

In [30]:
len(results['results']['bindings'])

1048576

imagine a matrix of field x institution/author, with the cells of the matrix representing either counts of papers, or binary occurrences. What fields have a diversity of authors/institutions? what are more monopolistics? what are the consequences of the differences between these especially over time?

is there an association between data sharing and market concentration (e.g. hh index)?

also, look at what aaron clausett has been working on

http://www.semantic-web-journal.net/system/files/swj1495.pdf

group differences in "tenure rates, competency evaluations, remuneration, allocation of research facilities, and
grant competitions" (way, larremore & clauset, 2016)


plots to make:

prevalency of links to various data-sharing platforms over time (see the [google spreadsheet](https://docs.google.com/spreadsheets/d/14GSgOppnvjS-pxGz1__fEBuj3KUNgKB7ni5nukHT8Ps/edit#gid=2016420688) for links)

market concentration by...author? institution?

# Some people working in the bibliometric/meta science space that might be relevant:

## Aaron Clauset:
https://arxiv.org/abs/1612.08228

https://arxiv.org/abs/1804.02760

https://arxiv.org/abs/1805.09966

## Dani Bassett:
https://arxiv.org/abs/1810.10534

https://arxiv.org/abs/1804.06434

https://www.sciencedirect.com/science/article/pii/S105381191830778X?via%3Dihub

## Jevin West:
https://jevinwest.org/papers/West2013PLoSOne.pdf

https://jevinwest.org/papers/West2014EconInquiry.pdf

https://jevinwest.org/papers/Vilhena2014SocScience.pdf

https://jevinwest.org/papers/Larsen2014HICSS.pdf

https://jevinwest.org/papers/Portenoy2016WWW.pdf

https://jevinwest.org/papers/Kim2017asa.pdf

https://jevinwest.org/papers/Chow2017fisheries.pdf

https://jevinwest.org/papers/King2017socius.pdf

https://jevinwest.org/papers/Piwowar2018peerj.pdf


## Michael Farber
http://dbis.informatik.uni-freiburg.de/content/team/faerber/papers/arXiv_Dataset_BIR2019.pdf



## Other:
https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1005962

https://cdn.elifesciences.org/articles/32822/elife-32822-v2.pdf

https://academic.oup.com/bioinformatics/advance-article/doi/10.1093/bioinformatics/btz070/5305021

https://link.springer.com/article/10.1007/s11192-012-0900-9

https://link.springer.com/chapter/10.1007/978-3-319-67056-0_18

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5761855/

https://onlinelibrary.wiley.com/doi/full/10.1002/asi.23266

https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0000308

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2990274/

In [ ]:
# datasets
# arxiv (downloaded)
# https://www.ncbi.nlm.nih.gov/research/bionlp/APIs/BioC-PMC/

# Pulling arxiv papers

These arxiv papers are connected to the ma graph. See Michael Farber's paper for additional details

In [8]:
f = open('/Downloads/arxiv_v2.1/0704.0001.txt','w') 
print(f) 

FileNotFoundError: [Errno 2] No such file or directory: '/Downloads/arxiv_v2.1/0704.0001.txt'

# Get the pubmed papers

In [2]:
#bioc-pmc files
files_to_get = pd.read_table('/home/riddleta/ac_knowl/data/filelist.txt', sep='\s+')
files_to_get.shape

/home/riddleta/ac_knowl/ac_knowl/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: read_table is deprecated, use read_csv instead.
  


(598386, 4)

In [3]:
oa_comm_use_files = pd.read_csv('/home/riddleta/ac_knowl/data/oa_comm_use_file_list.txt', 
                                sep='\\t', engine='python', header=None, skiprows=1)
oa_comm_use_files.columns = ['File', 'ref', 'PMCID', 'PMID', 'license']
oa_comm_use_files.shape

(1479862, 5)

In [4]:
oa_files = pd.read_table('/home/riddleta/ac_knowl/data/oa_file_list.txt', 
                         sep='\\t', engine='python', header=None, skiprows=1)
oa_files.columns = ['File', 'ref', 'PMCID', 'PMID', 'license']
oa_files.shape

/home/riddleta/ac_knowl/ac_knowl/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: read_table is deprecated, use read_csv instead.
  


(2408932, 5)

In [5]:
unique_ids = pd.concat([oa_files['PMCID'], oa_comm_use_files['PMCID'], files_to_get['PMCID']]).unique()

In [6]:
def return_useful_data(r):
    git_hits = 0
    osf_hits = 0
    nda_hits = 0
    open_neuro = 0
    fmri = 0
    res = 0
    r_out = r.json()
    date = r_out['date']
    src = r_out['source']
    try:
        doi = r_out['documents'][0]['passages'][0]['infons']['article-id_doi']
    except:
        doi = None
    try:
        yr = r_out['documents'][0]['passages'][0]['infons']['year']
    except:
        yr = None
    if yr not in ['2008', '2009', '2010', '2011', '2012', '2013', 
                  '2014', '2015', '2016', '2017', '2018', '2019']:
        return(date, src, doi, git_hits, osf_hits, nda_hits, open_neuro, fmri, res, yr)
    else:
        for passage in r_out['documents'][0]['passages']:
            if re.search(re.compile('github'), passage['text'].lower()):
                git_hits = git_hits + 1
            if re.search(re.compile('osf\.io'), passage['text'].lower()):
                osf_hits = osf_hits + 1
            if re.search(re.compile('nda\.nih\.gov'), passage['text'].lower()):
                nda_hits = nda_hits + 1
            if re.search(re.compile('openneuro'), passage['text'].lower()):
                open_neuro = open_neuro + 1
            if re.search(re.compile('fmri'), passage['text'].lower()):
                fmri = fmri + 1
            if re.search(re.compile('results'), passage['text'].lower()):
                res = res + 1
        
        return(date, src, doi, git_hits, osf_hits, nda_hits, open_neuro, fmri, res, yr)

In [134]:
git_links = []
for i, j in enumerate(unique_ids):
    pth = 'https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_json/' + j + '/unicode'
    r = requests.get(pth)
    git_links.append(return_useful_data(r) + (j, i))
    if i==10000:
        break

In [138]:
df = pd.DataFrame(git_links, columns=['date', 'src', 'doi', 'git_hits', 
                                      'osf_hits', 'nda_hits', 'open_neuro', 
                                      'fmri', 'res', 'yr', 'pmcid', 'idx'])
df.to_csv('/home/riddleta/ac_knowl/output/bionlp_10k.csv')

In [141]:
git_links = []
for i, j in enumerate(unique_ids[10000:]):
    pth = 'https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_json/' + j + '/unicode'
    r = requests.get(pth)
    git_links.append(return_useful_data(r) + (j, i))
    if i==15000:
        break

In [144]:
df = pd.DataFrame(git_links, columns=['date', 'src', 'doi', 'git_hits', 
                                      'osf_hits', 'nda_hits', 'open_neuro', 
                                      'fmri', 'res', 'yr', 'pmcid', 'idx'])
df.to_csv('/home/riddleta/ac_knowl/output/bionlp_02.csv')

In [14]:
git_links = []
missed = []

for i, j in enumerate(unique_ids[25000:]):
    pth = 'https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_json/' + j + '/unicode'
    try:
        r = requests.get(pth)
        git_links.append(return_useful_data(r) + (j, i))
    except:
        missed.append([i,j])
    if i==20000:
        break
df = pd.DataFrame(git_links, columns=['date', 'src', 'doi', 'git_hits', 
                                      'osf_hits', 'nda_hits', 'open_neuro', 
                                      'fmri', 'res', 'yr', 'pmcid', 'idx'])
df.to_csv('/home/riddleta/ac_knowl/output/bionlp_03.csv')

In [12]:
git_links = []
missed = []

progress = [i for i in range(0, 25000, 1000)]

for i, j in enumerate(unique_ids[45000:]):
    if i in progress:
        print(i)
    pth = 'https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_json/' + j + '/unicode'
    try:
        r = requests.get(pth)
        git_links.append(return_useful_data(r) + (j, i))
    except:
        missed.append([i,j])
    if i==25000:
        break


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000


In [16]:
git_links = []
missed = []

progress = [i for i in range(0, 30000, 1000)]

for i, j in enumerate(unique_ids[70000:]):
    if i in progress:
        print(i)
    pth = 'https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_json/' + j + '/unicode'
    try:
        r = requests.get(pth)
        git_links.append(return_useful_data(r) + (j, i))
    except:
        missed.append([i,j])
    if i==30000:
        break

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000


In [18]:
df = pd.DataFrame(git_links, columns=['date', 'src', 'doi', 'git_hits', 
                                      'osf_hits', 'nda_hits', 'open_neuro', 
                                      'fmri', 'res', 'yr', 'pmcid', 'idx'])
df.to_csv('/home/riddleta/ac_knowl/output/bionlp_05.csv')

In [20]:
git_links = []
missed = []

progress = [i for i in range(0, 100000, 2500)]

for i, j in enumerate(unique_ids[100000:]):
    if i in progress:
        print(i)
    pth = 'https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_json/' + j + '/unicode'
    try:
        r = requests.get(pth)
        git_links.append(return_useful_data(r) + (j, i))
    except:
        missed.append([i,j])
    if i==100000:
        break

0
2500
5000
7500
10000
12500
15000
17500
20000
22500
25000
27500
30000
32500
35000
37500
40000
42500
45000
47500
50000
52500
55000
57500
60000
62500
65000
67500
70000
72500
75000
77500
80000
82500
85000
87500
90000
92500
95000
97500


In [22]:
df = pd.DataFrame(git_links, columns=['date', 'src', 'doi', 'git_hits', 
                                      'osf_hits', 'nda_hits', 'open_neuro', 
                                      'fmri', 'res', 'yr', 'pmcid', 'idx'])
df.to_csv('/home/riddleta/ac_knowl/output/bionlp_06.csv')
df.head()

,date,src,doi,git_hits,osf_hits,nda_hits,open_neuro,fmri,res,yr,pmcid,idx
0,20181110,PMC,10.1083/jcb.200211071581251,0,0,0,0,0,0,2002,PMC2173096,0
1,20181110,PMC,10.1083/jcb.200208058,0,0,0,0,0,0,2002,PMC2173097,1
2,20181110,PMC,10.1083/jcb.200204153,0,0,0,0,0,0,2002,PMC2173098,2
3,20181110,PMC,10.1083/jcb.200206024,0,0,0,0,0,0,2002,PMC2173099,3
4,20181117,PMC,10.1083/jcb.200204102,0,0,0,0,0,0,2002,PMC2173100,4


In [24]:
git_links = []
missed = []

progress = [i for i in range(0, 35000, 2500)]

for i, j in enumerate(unique_ids[200000:]):
    if i in progress:
        print(i)
    pth = 'https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_json/' + j + '/unicode'
    try:
        r = requests.get(pth)
        git_links.append(return_useful_data(r) + (j, i))
    except:
        missed.append([i,j])
    if i==35000:
        break

0
2500
5000
7500
10000
12500
15000
17500
20000
22500
25000
27500
30000
32500


In [25]:
df = pd.DataFrame(git_links, columns=['date', 'src', 'doi', 'git_hits', 
                                      'osf_hits', 'nda_hits', 'open_neuro', 
                                      'fmri', 'res', 'yr', 'pmcid', 'idx'])
df.to_csv('/home/riddleta/ac_knowl/output/bionlp_07.csv')
df.head()

,date,src,doi,git_hits,osf_hits,nda_hits,open_neuro,fmri,res,yr,pmcid,idx
0,20181207,PMC,10.1186/1297-9686-29-4-413,0,0,0,0,0,0,1997,PMC2708220,0
1,20181207,PMC,10.1186/1297-9686-29-4-427,0,0,0,0,0,0,1997,PMC2708221,1
2,20181207,PMC,10.1186/1297-9686-29-4-451,0,0,0,0,0,0,1997,PMC2708222,2
3,20181207,PMC,10.1186/1297-9686-29-4-469,0,0,0,0,0,0,1997,PMC2708223,3
4,20181207,PMC,10.1186/1297-9686-29-4-483,0,0,0,0,0,0,1997,PMC2708224,4


In [27]:
git_links = []
progress = [i for i in range(0, 40000, 2500)]

for i, j in enumerate(unique_ids[235000:]):
    if i in progress:
        print(i)
    pth = 'https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_json/' + j + '/unicode'
    try:
        r = requests.get(pth)
        git_links.append(return_useful_data(r) + (j, i))
    except:
        missed.append([i,j])
    if i==40000:
        break

0
2500
5000
7500
10000
12500
15000
17500
20000
22500
25000
27500
30000
32500
35000
37500


In [29]:
df = pd.DataFrame(git_links, columns=['date', 'src', 'doi', 'git_hits', 
                                      'osf_hits', 'nda_hits', 'open_neuro', 
                                      'fmri', 'res', 'yr', 'pmcid', 'idx'])
df.to_csv('/home/riddleta/ac_knowl/output/bionlp_08.csv')
df.head()

,date,src,doi,git_hits,osf_hits,nda_hits,open_neuro,fmri,res,yr,pmcid,idx
0,20181124,PMC,10.4103/0970-2113.44123,0,0,0,0,0,1,2008,PMC2822325,0
1,20181207,PMC,10.4103/0970-2113.44127,0,0,0,0,0,1,2008,PMC2822326,1
2,20181124,PMC,10.4103/0970-2113.59590,0,0,0,0,0,0,2008,PMC2822327,2
3,20181124,PMC,10.4103/0970-2113.59591,0,0,0,0,0,2,2008,PMC2822328,3
4,20181124,PMC,10.4103/0970-2113.44122,0,0,0,0,0,6,2008,PMC2822329,4


In [30]:
git_links = []
progress = [i for i in range(0, 125000, 2500)]

for i, j in enumerate(unique_ids[275000:]):
    if i in progress:
        print(i)
    pth = 'https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_json/' + j + '/unicode'
    try:
        r = requests.get(pth)
        git_links.append(return_useful_data(r) + (j, i))
    except:
        missed.append([i,j])
    if i==125000:
        break

0
2500
5000
7500
10000
12500
15000
17500
20000
22500
25000
27500
30000
32500
35000
37500
40000
42500
45000
47500
50000
52500
55000
57500
60000
62500
65000
67500
70000
72500
75000
77500
80000
82500
85000
87500
90000
92500
95000
97500
100000
102500
105000
107500
110000
112500
115000
117500
120000
122500


In [32]:
df = pd.DataFrame(git_links, columns=['date', 'src', 'doi', 'git_hits', 
                                      'osf_hits', 'nda_hits', 'open_neuro', 
                                      'fmri', 'res', 'yr', 'pmcid', 'idx'])
df.to_csv('/home/riddleta/ac_knowl/output/bionlp_09.csv')
df.head()

,date,src,doi,git_hits,osf_hits,nda_hits,open_neuro,fmri,res,yr,pmcid,idx
0,20181121,PMC,10.1186/gm171,0,0,0,0,0,6,2011,PMC2945007,0
1,20181121,PMC,10.1186/gm172,0,0,0,0,0,2,2010,PMC2945008,1
2,20181120,PMC,10.1186/gm173,0,0,0,0,0,0,2011,PMC2945009,2
3,20181121,PMC,10.1186/gm174,0,0,0,0,0,5,2011,PMC2945010,3
4,20181121,PMC,10.1186/gm175,0,0,0,0,0,1,2011,PMC2945011,4


In [8]:
git_links = []
progress = [i for i in range(0, 45000, 2500)]

for i, j in enumerate(unique_ids[400000:]):
    if i in progress:
        print(i)
    pth = 'https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_json/' + j + '/unicode'
    try:
        r = requests.get(pth)
        git_links.append(return_useful_data(r) + (j, i))
    except:
        missed.append([i,j])
    if i==45000:
        break

0
2500
5000
7500
10000
12500
15000
17500
20000
22500
25000
27500
30000
32500
35000
37500
40000
42500


In [10]:
df = pd.DataFrame(git_links, columns=['date', 'src', 'doi', 'git_hits', 
                                      'osf_hits', 'nda_hits', 'open_neuro', 
                                      'fmri', 'res', 'yr', 'pmcid', 'idx'])
df.to_csv('/home/riddleta/ac_knowl/output/bionlp_10.csv')
df.head()

,date,src,doi,git_hits,osf_hits,nda_hits,open_neuro,fmri,res,yr,pmcid,idx
0,20181207,PMC,10.1186/1471-2105-12-420,0,0,0,0,0,5,2011,PMC3225379,0
1,20181110,PMC,10.1371/journal.pone.0027740,0,0,0,0,0,12,2011,PMC3225380,1
2,20181130,PMC,10.1371/journal.pone.0028098,0,0,0,0,0,13,2011,PMC3225381,2
3,20181110,PMC,10.1371/journal.pone.0027946,0,0,0,0,0,9,2011,PMC3225382,3
4,20181121,PMC,10.1371/journal.pone.0028053,0,0,0,0,0,6,2011,PMC3225383,4


In [11]:
git_links = []
missed = []
progress = [i for i in range(0, 20000, 2500)]

for i, j in enumerate(unique_ids[445000:]):
    if i in progress:
        print(i)
    pth = 'https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_json/' + j + '/unicode'
    try:
        r = requests.get(pth)
        git_links.append(return_useful_data(r) + (j, i))
    except:
        missed.append([i,j])
    if i==20000:
        break

0
2500
5000
7500
10000
12500
15000
17500


In [14]:
df = pd.DataFrame(git_links, columns=['date', 'src', 'doi', 'git_hits', 
                                      'osf_hits', 'nda_hits', 'open_neuro', 
                                      'fmri', 'res', 'yr', 'pmcid', 'idx'])
df.to_csv('/home/riddleta/ac_knowl/output/bionlp_11.csv')
df.head()

,date,src,doi,git_hits,osf_hits,nda_hits,open_neuro,fmri,res,yr,pmcid,idx
0,20181123,PMC,10.1186/1471-2210-8-S1-A54,0,0,0,0,0,1,2008,PMC3313246,0
1,20181201,PMC,10.1186/1471-2210-8-S1-A55,0,0,0,0,0,1,2008,PMC3313247,1
2,20181201,PMC,10.1186/1471-2210-8-S1-A56,0,0,0,0,0,2,2008,PMC3313248,2
3,20181201,PMC,10.1186/1471-2210-8-S1-A57,0,0,0,0,0,1,2008,PMC3313249,3
4,20181207,PMC,10.1186/1471-2210-8-S1-A58,0,0,0,0,0,1,2008,PMC3313250,4


In [15]:
git_links = []
missed = []
progress = [i for i in range(0, 10000, 2500)]

for i, j in enumerate(unique_ids[465000:]):
    if i in progress:
        print(i)
    pth = 'https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_json/' + j + '/unicode'
    try:
        r = requests.get(pth)
        git_links.append(return_useful_data(r) + (j, i))
    except:
        missed.append([i,j])
    if i==10000:
        break

0
2500
5000
7500


In [17]:
df = pd.DataFrame(git_links, columns=['date', 'src', 'doi', 'git_hits', 
                                      'osf_hits', 'nda_hits', 'open_neuro', 
                                      'fmri', 'res', 'yr', 'pmcid', 'idx'])
df.to_csv('/home/riddleta/ac_knowl/output/bionlp_12.csv')
df.head()

,date,src,doi,git_hits,osf_hits,nda_hits,open_neuro,fmri,res,yr,pmcid,idx
0,20181121,PMC,10.1371/journal.pgen.1002706,0,0,0,0,0,9,2012,PMC3349741,0
1,20181121,PMC,10.1371/journal.pgen.1002705,0,0,0,0,0,7,2012,PMC3349742,1
2,20181121,PMC,10.1371/journal.ppat.1002607,0,0,0,0,0,2,2012,PMC3349743,2
3,20190113,PMC,10.1371/journal.ppat.1002679,0,0,0,0,0,8,2012,PMC3349744,3
4,20181114,PMC,10.1371/journal.ppat.1002675,0,0,0,0,0,20,2012,PMC3349745,4


In [18]:
git_links = []
missed = []
progress = [i for i in range(0, 150000, 5000)]

for i, j in enumerate(unique_ids[475000:]):
    if i in progress:
        print(i)
    pth = 'https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_json/' + j + '/unicode'
    try:
        r = requests.get(pth)
        git_links.append(return_useful_data(r) + (j, i))
    except:
        missed.append([i,j])
    if i==150000:
        break

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000
145000


In [20]:
df = pd.DataFrame(git_links, columns=['date', 'src', 'doi', 'git_hits', 
                                      'osf_hits', 'nda_hits', 'open_neuro', 
                                      'fmri', 'res', 'yr', 'pmcid', 'idx'])
df.to_csv('/home/riddleta/ac_knowl/output/bionlp_13.csv')
df.head()

,date,src,doi,git_hits,osf_hits,nda_hits,open_neuro,fmri,res,yr,pmcid,idx
0,20181124,PMC,10.3201/eid1112.041293,0,0,0,0,0,0,2005,PMC3367621,0
1,20181207,PMC,10.3201/eid1112.050079,0,0,0,0,0,0,2005,PMC3367622,1
2,20181207,PMC,10.3201/eid1112.050158,0,0,0,0,0,0,2005,PMC3367623,2
3,20181124,PMC,10.3201/eid1112.050217,0,0,0,0,0,0,2005,PMC3367624,3
4,20181207,PMC,10.3201/eid1112.050220,0,0,0,0,0,0,2005,PMC3367625,4


In [23]:
git_links = []
missed = []
progress = [i for i in range(0, 15000, 2500)]

for i, j in enumerate(unique_ids[625000:]):
    if i in progress:
        print(i)
    pth = 'https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_json/' + j + '/unicode'
    try:
        r = requests.get(pth)
        git_links.append(return_useful_data(r) + (j, i))
    except:
        missed.append([i,j])
    if i==15000:
        break

0
2500
5000
7500
10000
12500


In [25]:
df = pd.DataFrame(git_links, columns=['date', 'src', 'doi', 'git_hits', 
                                      'osf_hits', 'nda_hits', 'open_neuro', 
                                      'fmri', 'res', 'yr', 'pmcid', 'idx'])
df.to_csv('/home/riddleta/ac_knowl/output/bionlp_14.csv')
df.head()

,date,src,doi,git_hits,osf_hits,nda_hits,open_neuro,fmri,res,yr,pmcid,idx
0,20181203,PMC,10.3341/kjo.2013.27.3.211,0,0,0,0,0,2,2013,PMC3663066,0
1,20181202,PMC,10.3341/kjo.2013.27.3.215,0,0,0,0,0,0,2013,PMC3663067,1
2,20181206,PMC,10.3341/kjo.2013.27.3.219,0,0,0,0,0,1,2013,PMC3663068,2
3,20181203,PMC,10.3341/kjo.2013.27.3.224,0,0,0,0,0,4,2013,PMC3663069,3
4,20181113,PMC,10.1021/ja4020925,0,0,0,0,0,4,2013,PMC3663070,4


In [26]:
git_links = []
missed = []
progress = [i for i in range(0, 20000, 2500)]

for i, j in enumerate(unique_ids[640000:]):
    if i in progress:
        print(i)
    pth = 'https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_json/' + j + '/unicode'
    try:
        r = requests.get(pth)
        git_links.append(return_useful_data(r) + (j, i))
    except:
        missed.append([i,j])
    if i==20000:
        break

0
2500
5000
7500
10000
12500
15000
17500


In [28]:
df = pd.DataFrame(git_links, columns=['date', 'src', 'doi', 'git_hits', 
                                      'osf_hits', 'nda_hits', 'open_neuro', 
                                      'fmri', 'res', 'yr', 'pmcid', 'idx'])
df.to_csv('/home/riddleta/ac_knowl/output/bionlp_15.csv')
df.head()

,date,src,doi,git_hits,osf_hits,nda_hits,open_neuro,fmri,res,yr,pmcid,idx
0,20181207,PMC,10.3389/fpsyg.2013.00354,0,0,0,0,0,7,2013,PMC3691502,0
1,20181207,PMC,10.3389/fpsyg.2013.00370,0,0,0,0,0,9,2013,PMC3691503,1
2,20181207,PMC,10.3389/fnhum.2013.00304,0,0,0,0,2,5,2013,PMC3691504,2
3,20181210,PMC,10.3389/fmicb.2013.00161,0,0,0,0,0,10,2013,PMC3691505,3
4,20181210,PMC,10.3389/fnsys.2013.00020,0,0,0,0,1,2,2013,PMC3691506,4


In [29]:
git_links = []
missed = []
progress = [i for i in range(0, 25000, 2500)]

for i, j in enumerate(unique_ids[660000:]):
    if i in progress:
        print(i)
    pth = 'https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_json/' + j + '/unicode'
    try:
        r = requests.get(pth)
        git_links.append(return_useful_data(r) + (j, i))
    except:
        missed.append([i,j])
    if i==25000:
        break

0
2500
5000
7500
10000
12500
15000
17500
20000
22500


In [31]:
df = pd.DataFrame(git_links, columns=['date', 'src', 'doi', 'git_hits', 
                                      'osf_hits', 'nda_hits', 'open_neuro', 
                                      'fmri', 'res', 'yr', 'pmcid', 'idx'])
df.to_csv('/home/riddleta/ac_knowl/output/bionlp_16.csv')
df.head()

,date,src,doi,git_hits,osf_hits,nda_hits,open_neuro,fmri,res,yr,pmcid,idx
0,20181121,PMC,10.1371/journal.pone.0069952,0,0,0,0,0,9,2013,PMC3732294,0
1,20181121,PMC,10.1371/journal.pone.0069785,0,0,0,0,0,5,2013,PMC3732295,1
2,20181121,PMC,10.1371/journal.pone.0069488,0,0,0,0,0,6,2013,PMC3732296,2
3,20181206,PMC,10.4161/biom.23284,0,0,0,0,0,3,2013,PMC3732320,3
4,20181206,PMC,10.4161/biom.23799,0,0,0,0,0,9,2013,PMC3732321,4


In [32]:
git_links = []
missed = []
progress = [i for i in range(0, 30000, 2500)]

for i, j in enumerate(unique_ids[685000:]):
    if i in progress:
        print(i)
    pth = 'https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_json/' + j + '/unicode'
    try:
        r = requests.get(pth)
        git_links.append(return_useful_data(r) + (j, i))
    except:
        missed.append([i,j])
    if i==30000:
        break

0
2500
5000
7500
10000
12500
15000
17500
20000
22500
25000
27500


In [34]:
df = pd.DataFrame(git_links, columns=['date', 'src', 'doi', 'git_hits', 
                                      'osf_hits', 'nda_hits', 'open_neuro', 
                                      'fmri', 'res', 'yr', 'pmcid', 'idx'])
df.to_csv('/home/riddleta/ac_knowl/output/bionlp_17.csv')
df.head()

,date,src,doi,git_hits,osf_hits,nda_hits,open_neuro,fmri,res,yr,pmcid,idx
0,20181202,PMC,10.1155/2013/347242,0,0,0,0,0,2,2013,PMC3782758,0
1,20181202,PMC,10.1155/2013/196409,0,0,0,0,0,7,2013,PMC3782759,1
2,20181112,PMC,10.1155/2013/278978,0,0,0,0,0,6,2013,PMC3782760,2
3,20181112,PMC,10.1155/2013/383685,0,0,0,0,0,2,2013,PMC3782761,3
4,20181113,PMC,10.1155/2013/132759,0,0,0,0,0,15,2013,PMC3782762,4


In [8]:
git_links = []
missed = []
progress = [i for i in range(0, 175000, 5000)]

for i, j in enumerate(unique_ids[715000:]):
    if i in progress:
        print(i)
    pth = 'https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_json/' + j + '/unicode'
    try:
        r = requests.get(pth)
        git_links.append(return_useful_data(r) + (j, i))
    except:
        missed.append([i,j])
    if i==175000:
        break

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000
145000
150000
155000
160000
165000
170000


In [10]:
df = pd.DataFrame(git_links, columns=['date', 'src', 'doi', 'git_hits', 
                                      'osf_hits', 'nda_hits', 'open_neuro', 
                                      'fmri', 'res', 'yr', 'pmcid', 'idx'])
df.to_csv('/home/riddleta/ac_knowl/output/bionlp_18.csv')
df.head()

,date,src,doi,git_hits,osf_hits,nda_hits,open_neuro,fmri,res,yr,pmcid,idx
0,20181124,PMC,10.1155/2013/436932,0,0,0,0,0,3,2013,PMC3842064,0
1,20181124,PMC,10.1155/2013/791918,0,0,0,0,0,8,2013,PMC3842065,1
2,20181124,PMC,10.1155/2013/924276,0,0,0,0,0,5,2013,PMC3842066,2
3,20181124,PMC,10.1155/2013/367283,0,0,0,0,0,15,2013,PMC3842067,3
4,20181124,PMC,10.1155/2013/731346,0,0,0,0,0,3,2013,PMC3842068,4


In [11]:
git_links = []
missed = []
progress = [i for i in range(0, 35000, 2500)]

for i, j in enumerate(unique_ids[890000:]):
    if i in progress:
        print(i)
    pth = 'https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_json/' + j + '/unicode'
    try:
        r = requests.get(pth)
        git_links.append(return_useful_data(r) + (j, i))
    except:
        missed.append([i,j])
    if i==35000:
        break

0
2500
5000
7500
10000
12500
15000
17500
20000
22500
25000
27500
30000
32500


In [12]:
df = pd.DataFrame(git_links, columns=['date', 'src', 'doi', 'git_hits', 
                                      'osf_hits', 'nda_hits', 'open_neuro', 
                                      'fmri', 'res', 'yr', 'pmcid', 'idx'])
df.to_csv('/home/riddleta/ac_knowl/output/bionlp_19.csv')
df.head()

,date,src,doi,git_hits,osf_hits,nda_hits,open_neuro,fmri,res,yr,pmcid,idx
0,20181210,PMC,10.5090/kjtcs.2014.47.3.317,0,0,0,0,0,2,2014,PMC4157490,0
1,20181113,PMC,10.5090/kjtcs.2014.47.3.320,0,0,0,0,0,0,2014,PMC4157491,1
2,20181113,PMC,10.5090/kjtcs.2014.47.3.325,0,0,0,0,0,0,2014,PMC4157492,2
3,20181113,PMC,10.1177/2048004014539666,0,0,0,0,0,2,2014,PMC4157493,3
4,20181114,PMC,10.5090/kjtcs.2014.47.4.327,0,0,0,0,0,3,2014,PMC4157494,4


In [13]:
git_links = []
missed = []
progress = [i for i in range(0, 30000, 2500)]

for i, j in enumerate(unique_ids[925000:]):
    if i in progress:
        print(i)
    pth = 'https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_json/' + j + '/unicode'
    try:
        r = requests.get(pth)
        git_links.append(return_useful_data(r) + (j, i))
    except:
        missed.append([i,j])
    if i==30000:
        break

0
2500
5000
7500
10000
12500
15000
17500
20000
22500
25000
27500


In [15]:
df = pd.DataFrame(git_links, columns=['date', 'src', 'doi', 'git_hits', 
                                      'osf_hits', 'nda_hits', 'open_neuro', 
                                      'fmri', 'res', 'yr', 'pmcid', 'idx'])
df.to_csv('/home/riddleta/ac_knowl/output/bionlp_20.csv')
df.head()

,date,src,doi,git_hits,osf_hits,nda_hits,open_neuro,fmri,res,yr,pmcid,idx
0,20181123,PMC,10.1371/journal.pone.0113017,0,0,0,0,0,0,2014,PMC4218860,0
1,20181115,PMC,10.1371/journal.pone.0109125,0,0,0,0,0,8,2014,PMC4218861,1
2,20181115,PMC,10.1371/journal.pone.0109700,0,0,0,0,1,18,2014,PMC4218862,2
3,20181120,PMC,10.1113/expphysiol.2014.081190,0,0,0,0,0,5,2014,PMC4218865,3
4,20181115,PMC,10.1021/mp5003043,0,0,0,0,0,9,2014,PMC4218868,4


In [8]:
git_links = []
missed = []
progress = [i for i in range(0, 35000, 10000)]

for i, j in enumerate(unique_ids[955000:]):
    if i in progress:
        print(i)
    pth = 'https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_json/' + j + '/unicode'
    try:
        r = requests.get(pth)
        git_links.append(return_useful_data(r) + (j, i))
    except:
        missed.append([i,j])
    if i==35000:
        break

0
10000
20000
30000


In [10]:
df = pd.DataFrame(git_links, columns=['date', 'src', 'doi', 'git_hits', 
                                      'osf_hits', 'nda_hits', 'open_neuro', 
                                      'fmri', 'res', 'yr', 'pmcid', 'idx'])
df.to_csv('/home/riddleta/ac_knowl/output/bionlp_21.csv')
df.head()

,date,src,doi,git_hits,osf_hits,nda_hits,open_neuro,fmri,res,yr,pmcid,idx
0,20181124,PMC,10.5812/ircmj.15308,0,0,0,0,0,10,2014,PMC4270654,0
1,20181124,PMC,10.5812/ircmj.21835,0,0,0,0,0,3,2014,PMC4270655,1
2,20181124,PMC,10.5812/ircmj.15312,0,0,0,0,0,4,2014,PMC4270656,2
3,20181123,PMC,10.5812/ircmj.14919,0,0,0,0,0,2,2014,PMC4270657,3
4,20181124,PMC,10.5812/ircmj.15164,0,0,0,0,0,0,2014,PMC4270658,4


In [11]:
git_links = []
missed = []
progress = [i for i in range(0, 25000, 5000)]

for i, j in enumerate(unique_ids[990000:]):
    if i in progress:
        print(i)
    pth = 'https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_json/' + j + '/unicode'
    try:
        r = requests.get(pth)
        git_links.append(return_useful_data(r) + (j, i))
    except:
        missed.append([i,j])
    if i==25000:
        break

0
5000
10000
15000
20000


In [13]:
df = pd.DataFrame(git_links, columns=['date', 'src', 'doi', 'git_hits', 
                                      'osf_hits', 'nda_hits', 'open_neuro', 
                                      'fmri', 'res', 'yr', 'pmcid', 'idx'])
df.to_csv('/home/riddleta/ac_knowl/output/bionlp_22.csv')
df.head()

,date,src,doi,git_hits,osf_hits,nda_hits,open_neuro,fmri,res,yr,pmcid,idx
0,20181201,PMC,10.4103/0973-1075.150164,0,0,0,0,0,0,2015,PMC4332122,0
1,20181123,PMC,10.4103/0973-1075.150170,0,0,0,0,0,4,2015,PMC4332123,1
2,20181201,PMC,10.4103/0973-1075.150148,0,0,0,0,0,2,2015,PMC4332124,2
3,20181201,PMC,10.4103/0973-1075.150172,0,0,0,0,0,2,2015,PMC4332125,3
4,20181201,PMC,10.4103/0973-1075.150175,0,0,0,0,0,3,2015,PMC4332126,4


In [7]:
git_links = []
missed = []
progress = [i for i in range(0, 200000, 10000)]

for i, j in enumerate(unique_ids[1015000:]):
    if i in progress:
        print(i)
    pth = 'https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_json/' + j + '/unicode'
    try:
        r = requests.get(pth)
        git_links.append(return_useful_data(r) + (j, i))
    except:
        missed.append([i,j])
    if i==200000:
        break

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000


In [9]:
df = pd.DataFrame(git_links, columns=['date', 'src', 'doi', 'git_hits', 
                                      'osf_hits', 'nda_hits', 'open_neuro', 
                                      'fmri', 'res', 'yr', 'pmcid', 'idx'])
df.to_csv('/home/riddleta/ac_knowl/output/bionlp_23.csv')
df.head()

,date,src,doi,git_hits,osf_hits,nda_hits,open_neuro,fmri,res,yr,pmcid,idx
0,20190105,PMC,10.1080/14780887.2014.948697,0,0,0,0,0,0,2015,PMC4376240,0
1,20181121,PMC,10.1080/14427591.2014.882250,0,0,0,0,0,1,2014,PMC4376241,1
2,20190105,PMC,10.1080/15299732.2015.995021,0,0,0,0,0,0,2015,PMC4376242,2
3,20181121,PMC,10.1002/sia.5429,0,0,0,0,0,8,2014,PMC4376248,3
4,20181121,PMC,10.1002/sia.5376,0,0,0,0,0,5,2014,PMC4376249,4


In [11]:
git_links = []
missed = []
progress = [i for i in range(0, 40000, 10000)]

for i, j in enumerate(unique_ids[1215000:]):
    if i in progress:
        print(i)
    pth = 'https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_json/' + j + '/unicode'
    try:
        r = requests.get(pth)
        git_links.append(return_useful_data(r) + (j, i))
    except:
        missed.append([i,j])
    if i==40000:
        break

0
10000
20000
30000


In [13]:
df = pd.DataFrame(git_links, columns=['date', 'src', 'doi', 'git_hits', 
                                      'osf_hits', 'nda_hits', 'open_neuro', 
                                      'fmri', 'res', 'yr', 'pmcid', 'idx'])
df.to_csv('/home/riddleta/ac_knowl/output/bionlp_24.csv')
df.head()

,date,src,doi,git_hits,osf_hits,nda_hits,open_neuro,fmri,res,yr,pmcid,idx
0,20181130,PMC,10.1038/nature14984,0,0,0,0,0,7,2016,PMC4713039,0
1,20181112,PMC,None,0,0,0,0,0,0,2016,PMC4713040,1
2,20181112,PMC,10.1038/srep19264,0,0,0,0,0,4,2016,PMC4713048,2
3,20181112,PMC,10.1038/srep19260,0,0,0,0,0,4,2016,PMC4713049,3
4,20181207,PMC,10.1038/srep19256,0,0,0,0,0,16,2016,PMC4713050,4


In [14]:
git_links = []
missed = []
progress = [i for i in range(0, 45000, 10000)]

for i, j in enumerate(unique_ids[1255000:]):
    if i in progress:
        print(i)
    pth = 'https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_json/' + j + '/unicode'
    try:
        r = requests.get(pth)
        git_links.append(return_useful_data(r) + (j, i))
    except:
        missed.append([i,j])
    if i==45000:
        break

0
10000
20000
30000
40000


In [16]:
df = pd.DataFrame(git_links, columns=['date', 'src', 'doi', 'git_hits', 
                                      'osf_hits', 'nda_hits', 'open_neuro', 
                                      'fmri', 'res', 'yr', 'pmcid', 'idx'])
df.to_csv('/home/riddleta/ac_knowl/output/bionlp_25.csv')
df.head()

,date,src,doi,git_hits,osf_hits,nda_hits,open_neuro,fmri,res,yr,pmcid,idx
0,20181121,PMC,10.1186/s40064-016-1758-8,0,0,0,0,0,1,2016,PMC4777971,0
1,20190111,PMC,10.1186/s40064-016-1933-y,0,0,0,0,0,1,2016,PMC4777974,1
2,20181121,PMC,10.1186/s40064-016-1890-5,0,0,0,0,0,3,2016,PMC4777976,2
3,20181121,PMC,10.1186/s40064-016-1774-8,0,0,0,0,0,9,2016,PMC4777977,3
4,20181204,PMC,10.1186/s13613-016-0121-0,0,0,0,0,0,3,2016,PMC4777978,4


In [17]:
git_links = []
missed = []
progress = [i for i in range(0, 225000, 10000)]

for i, j in enumerate(unique_ids[1300000:]):
    if i in progress:
        print(i)
    pth = 'https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_json/' + j + '/unicode'
    try:
        r = requests.get(pth)
        git_links.append(return_useful_data(r) + (j, i))
    except:
        missed.append([i,j])
    if i==225000:
        break

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000


In [18]:
df = pd.DataFrame(git_links, columns=['date', 'src', 'doi', 'git_hits', 
                                      'osf_hits', 'nda_hits', 'open_neuro', 
                                      'fmri', 'res', 'yr', 'pmcid', 'idx'])
df.to_csv('/home/riddleta/ac_knowl/output/bionlp_25.csv')
df.head()

,date,src,doi,git_hits,osf_hits,nda_hits,open_neuro,fmri,res,yr,pmcid,idx
0,20190218,PMC,10.1371/journal.pone.0149955,0,0,0,0,0,5,2016,PMC4854442,0
1,20190218,PMC,10.1371/journal.pone.0154629,0,0,0,0,0,6,2016,PMC4854444,1
2,20190219,PMC,10.1371/journal.pone.0154324,3,0,0,0,0,6,2016,PMC4854445,2
3,20181124,PMC,10.1016/j.jhealeco.2015.09.007,0,0,0,0,0,19,2016,PMC4854446,3
4,20190218,PMC,10.1371/journal.pone.0154763,0,0,0,0,0,3,2016,PMC4854447,4


In [7]:
git_links = []
missed = []
progress = [i for i in range(0, 50000, 10000)]

for i, j in enumerate(unique_ids[1525000:]):
    if i in progress:
        print(i)
    pth = 'https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_json/' + j + '/unicode'
    try:
        r = requests.get(pth)
        git_links.append(return_useful_data(r) + (j, i))
    except:
        missed.append([i,j])
    if i==50000:
        break

0
10000
20000
30000
40000


In [9]:
df = pd.DataFrame(git_links, columns=['date', 'src', 'doi', 'git_hits', 
                                      'osf_hits', 'nda_hits', 'open_neuro', 
                                      'fmri', 'res', 'yr', 'pmcid', 'idx'])
df.to_csv('/home/riddleta/ac_knowl/output/bionlp_26.csv')
df.head()

,date,src,doi,git_hits,osf_hits,nda_hits,open_neuro,fmri,res,yr,pmcid,idx
0,20181207,PMC,None,0,0,0,0,0,0,1927,PMC5197919,0
1,20181111,PMC,None,0,0,0,0,0,0,1927,PMC5197920,1
2,20181123,PMC,10.1017/S0033291716002166,0,0,0,0,0,6,2016,PMC5197921,2
3,20181206,PMC,10.1017/S0963180116000712,0,0,0,0,0,0,2017,PMC5197922,3
4,20181206,PMC,10.1017/S0007114516003688,0,0,0,0,0,15,2016,PMC5197923,4


In [10]:
git_links = []
missed = []
progress = [i for i in range(0, 25000, 5000)]

for i, j in enumerate(unique_ids[1575000:]):
    if i in progress:
        print(i)
    pth = 'https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_json/' + j + '/unicode'
    try:
        r = requests.get(pth)
        git_links.append(return_useful_data(r) + (j, i))
    except:
        missed.append([i,j])
    if i==25000:
        break

0
5000
10000
15000
20000


In [12]:
df = pd.DataFrame(git_links, columns=['date', 'src', 'doi', 'git_hits', 
                                      'osf_hits', 'nda_hits', 'open_neuro', 
                                      'fmri', 'res', 'yr', 'pmcid', 'idx'])
df.to_csv('/home/riddleta/ac_knowl/output/bionlp_27.csv')
df.head()

,date,src,doi,git_hits,osf_hits,nda_hits,open_neuro,fmri,res,yr,pmcid,idx
0,20181115,PMC,None,0,0,0,0,0,0,1894,PMC5263745,0
1,20181115,PMC,None,0,0,0,0,0,0,1894,PMC5263747,1
2,20181116,PMC,None,0,0,0,0,0,0,1894,PMC5263748,2
3,20181116,PMC,None,0,0,0,0,0,0,1894,PMC5263749,3
4,20181106,PMC,None,0,0,0,0,0,0,1894,PMC5263750,4


In [13]:
git_links = []
missed = []
progress = [i for i in range(0, 250000, 10000)]

for i, j in enumerate(unique_ids[1600000:]):
    if i in progress:
        print(i)
    pth = 'https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_json/' + j + '/unicode'
    try:
        r = requests.get(pth)
        git_links.append(return_useful_data(r) + (j, i))
    except:
        missed.append([i,j])
    if i==250000:
        break

df = pd.DataFrame(git_links, columns=['date', 'src', 'doi', 'git_hits', 
                                      'osf_hits', 'nda_hits', 'open_neuro', 
                                      'fmri', 'res', 'yr', 'pmcid', 'idx'])
df.to_csv('/home/riddleta/ac_knowl/output/bionlp_28.csv')

git_links = []
missed = []
progress = [i for i in range(0, 250000, 10000)]

for i, j in enumerate(unique_ids[1850000:]):
    if i in progress:
        print(i)
    pth = 'https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_json/' + j + '/unicode'
    try:
        r = requests.get(pth)
        git_links.append(return_useful_data(r) + (j, i))
    except:
        missed.append([i,j])
    if i==250000:
        break

df = pd.DataFrame(git_links, columns=['date', 'src', 'doi', 'git_hits', 
                                      'osf_hits', 'nda_hits', 'open_neuro', 
                                      'fmri', 'res', 'yr', 'pmcid', 'idx'])
df.to_csv('/home/riddleta/ac_knowl/output/bionlp_29.csv')

git_links = []
missed = []
progress = [i for i in range(0, 250000, 10000)]

for i, j in enumerate(unique_ids[2125000:]):
    if i in progress:
        print(i)
    pth = 'https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_json/' + j + '/unicode'
    try:
        r = requests.get(pth)
        git_links.append(return_useful_data(r) + (j, i))
    except:
        missed.append([i,j])
    if i==250000:
        break

df = pd.DataFrame(git_links, columns=['date', 'src', 'doi', 'git_hits', 
                                      'osf_hits', 'nda_hits', 'open_neuro', 
                                      'fmri', 'res', 'yr', 'pmcid', 'idx'])
df.to_csv('/home/riddleta/ac_knowl/output/bionlp_30.csv')

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000


In [8]:
git_links = []
missed = []
progress = [i for i in range(0, 55000, 5000)]

for i, j in enumerate(unique_ids[2375000:]):
    if i in progress:
        print(i)
    pth = 'https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_json/' + j + '/unicode'
    try:
        r = requests.get(pth)
        git_links.append(return_useful_data(r) + (j, i))
    except:
        missed.append([i,j])
    if i==55000:
        break


0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000


In [10]:
df = pd.DataFrame(git_links, columns=['date', 'src', 'doi', 'git_hits', 
                                      'osf_hits', 'nda_hits', 'open_neuro', 
                                      'fmri', 'res', 'yr', 'pmcid', 'idx'])
df.to_csv('/home/riddleta/ac_knowl/output/bionlp_31.csv')
#df.head()

In [11]:
git_links = []
missed = []
progress = [i for i in range(0, 275000, 10000)]

for i, j in enumerate(unique_ids[2430000:]):
    if i in progress:
        print(i)
    pth = 'https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_json/' + j + '/unicode'
    try:
        r = requests.get(pth)
        git_links.append(return_useful_data(r) + (j, i))
    except:
        missed.append([i,j])
    if i==275000:
        break

df = pd.DataFrame(git_links, columns=['date', 'src', 'doi', 'git_hits', 
                                      'osf_hits', 'nda_hits', 'open_neuro', 
                                      'fmri', 'res', 'yr', 'pmcid', 'idx'])
df.to_csv('/home/riddleta/ac_knowl/output/bionlp_32.csv')

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000


In [12]:
git_links = []
missed = []
progress = [i for i in range(0, 275000, 10000)]

for i, j in enumerate(unique_ids[2704000:]):
    if i in progress:
        print(i)
    pth = 'https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_json/' + j + '/unicode'
    try:
        r = requests.get(pth)
        git_links.append(return_useful_data(r) + (j, i))
    except:
        missed.append([i,j])
    if i==275000:
        break

df = pd.DataFrame(git_links, columns=['date', 'src', 'doi', 'git_hits', 
                                      'osf_hits', 'nda_hits', 'open_neuro', 
                                      'fmri', 'res', 'yr', 'pmcid', 'idx'])
df.to_csv('/home/riddleta/ac_knowl/output/bionlp_33.csv')

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000


In [133]:
len(unique_ids)

2975022

In [150]:
tmpyr = '2011'
int(tmpyr)

2011

In [27]:
github_ex = re.compile('github')
re.search(github_ex, r_out['documents'][0]['passages'][0]['text'])

In [82]:
return_useful_data(r) + (j, i)

('20181121', 'PMC', 0, 0, 0, 0, 0, 'PMC126257', 350)

In [39]:
r_out['date']

'20181121'

In [57]:
r_out['documents'][0]['passages'][0]['text'].lower()

'resolution of airway inflammation and hyperreactivity after in vivo transfer of cd4+cd25+ regulatory t cells is interleukin 10 dependent'

In [62]:
temp_txt = 'resolution of airway inflammation and hyperreactivity after in vivo transfer of cd4+cd25+ regulatory t cells is interleukin 10 dependent. we also posted our data on github.com'



In [63]:
re.search(re.compile('github'), temp_txt.lower())

<re.Match object; span=(165, 171), match='github'>

In [13]:
r = requests.get('https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_json/PMC1350743/unicode')

In [14]:
r_out = r.json()

In [15]:
r_out.keys()

dict_keys(['date', 'source', 'infons', 'documents', 'key'])

In [16]:
r = requests.get('https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_json/PMC1249490/unicode')
r_out2 = r.json()

In [21]:
out_dat = [r_out, r_out2]
len(out_dat)

2

In [33]:
r_out['documents'][0].keys()

dict_keys(['passages', 'infons', 'id', 'relations'])

In [42]:
r_out['documents'][0]['passages'][0]

{'text': 'Resolution of airway inflammation and hyperreactivity after in vivo transfer of CD4+CD25+ regulatory T cells is interleukin 10 dependent',
 'offset': 0,
 'relations': [],
 'infons': {'name_2': 'surname:Robinson;given-names:Douglas S.',
  'name_3': 'surname:Lloyd;given-names:Clare M.',
  'name_0': 'surname:Kearley;given-names:Jennifer',
  'name_1': 'surname:Barker;given-names:Jane E.',
  'license': 'This article is distributed under the terms of an Attribution–Noncommercial–Share Alike–No Mirror Sites license for the first six months after the publication date (see http://www.rupress.org/terms). After six months it is available under a Creative Commons License (Attribution–Noncommercial–Share Alike 4.0 Unported license, as described at http://creativecommons.org/licenses/by-nc-sa/4.0/).',
  'article-id_publisher-id': '20051166',
  'issue': '11',
  'article-id_doi': '10.1084/jem.20051166',
  'section_type': 'TITLE',
  'article-id_pmid': '16314435',
  'lpage': '1547',
  'article

In [13]:
for passage in r_out['documents'][0]['passages']:
    print(passage['infons']['section_type'])

TITLE
ABSTRACT
INTRO
INTRO
INTRO
RESULTS
RESULTS
RESULTS
RESULTS
RESULTS
RESULTS
FIG
RESULTS
RESULTS
RESULTS
FIG
RESULTS
RESULTS
RESULTS
RESULTS
RESULTS
RESULTS
FIG
RESULTS
RESULTS
FIG
RESULTS
RESULTS
FIG
DISCUSS
DISCUSS
DISCUSS
DISCUSS
DISCUSS
DISCUSS
DISCUSS
DISCUSS
METHODS
METHODS
METHODS
METHODS
METHODS
METHODS
METHODS
METHODS
METHODS
METHODS
METHODS
METHODS
METHODS
METHODS
METHODS
METHODS
METHODS
METHODS
METHODS
METHODS
METHODS
METHODS
METHODS
METHODS
METHODS
METHODS
METHODS
METHODS
METHODS
METHODS
REF
REF
REF
REF
REF
REF
REF
REF
REF
REF
REF
REF
REF
REF
REF
REF
REF
REF
REF
REF
REF
REF
REF
REF
REF
REF
REF
REF
REF
REF
REF
REF
REF
REF
REF
REF
REF
REF
REF
REF
REF


In [ ]:
# affiliations https://www.ncbi.nlm.nih.gov/pmc/tools/get-metadata/